In [1]:
from math import sqrt
from numpy import concatenate
import matplotlib as mpl
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from keras.models import Sequential
from tensorflow.keras.models import load_model
from keras.layers import LSTM, Dense, Masking
from tensorflow.keras import callbacks as cb
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
from hydro_helpers.scoring import render_score
from hydro_helpers.reporting import cv_report
from hydro_helpers._utils import _nse as NSE
from hydro_helpers._utils import _rmse as RMSE
mpl.rcParams['figure.figsize'] = (20, 10)
mpl.rcParams['axes.grid'] = False
tf.random.set_seed(13)

Using TensorFlow backend.


In [4]:
river = 'lena'
df = pd.read_csv('data/lena/train_df.txt', index_col='date')
print(df.shape)
df.index = pd.DatetimeIndex(df.index)
# print(df.mean(axis=0))
df.head()

(12207, 211)


,stage_max_3019,stage_max_3021,stage_max_3024,stage_max_3027,stage_max_3028,stage_max_3029,stage_max_3030,stage_max_3031,stage_max_3032,stage_max_3035,...,prec_30372,prec_30385,prec_30393,prec_30433,prec_30471,prec_30493,prec_31004,prec_31026,prec_31102,prec_31137
date,,,,,,,,,,,,,,,,,,,,,
1985-01-01,-23.0,103.0,106.0,25.0,258.0,51,126.0,152.0,143,59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.2,0.0
1985-01-02,-23.0,103.0,106.0,23.0,258.0,49,122.0,150.0,141,58,...,0.0,0.0,0.0,0.9,0.0,0.0,0.5,0.7,0.0,0.7
1985-01-03,-23.0,102.0,106.0,21.0,255.0,47,120.0,147.0,140,57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.6,1.6,0.0,0.5
1985-01-04,-24.0,102.0,106.0,19.0,252.0,45,119.0,146.0,139,55,...,0.0,0.0,0.6,0.3,0.0,0.2,3.0,0.2,0.3,0.3
1985-01-05,-24.0,102.0,106.0,17.0,251.0,43,119.0,145.0,135,53,...,0.0,0.0,0.4,0.0,0.0,0.3,0.6,0.0,0.2,1.8


In [5]:
# run once
df = df.dropna(axis=0, how='all')
df = df.dropna(axis=1, how='all')
# print(df.isna().sum())
print(df.shape)
del df['stage_max_3048']
df = df.apply(lambda x: x.fillna(x.mean()),axis=0)
# print(df.isna().sum())
df = df.apply(lambda x: x.fillna(0),axis=0)
print(df.isna().sum())

(12207, 192)
stage_max_3019        0
stage_max_3021        0
stage_max_3024        0
stage_max_3027        0
stage_max_3028        0
stage_max_3029        0
stage_max_3030        0
stage_max_3031        0
stage_max_3032        0
stage_max_3035        0
stage_max_3036        0
stage_max_3037        0
stage_max_3038        0
stage_max_3041        0
stage_max_3042        0
stage_max_3045        0
stage_max_3047        0
stage_max_3050        0
stage_max_3087        0
stage_max_3106        0
stage_max_3169        0
stage_max_3180        0
stage_max_3229        0
stage_max_3230        0
stage_max_3554        0
stage_max_3555        0
ice_thickness_3019    0
ice_thickness_3021    0
ice_thickness_3024    0
ice_thickness_3027    0
                     ..
prec_24763            0
prec_24817            0
prec_24923            0
prec_24933            0
prec_24944            0
prec_24951            0
prec_24959            0
prec_24966            0
prec_24967            0
prec_30028            0
pre

In [ ]:
# df.plot(subplots=True, figsize=[20,20])
# plt.show()
corr = round(df.corr(), 2)
# целевые гидропосты
# rownames = [k for k in corr.columns[corr.columns.str.contains("3019|3027|3028|3029|3030|3035|3041|3045|3050|3230|3036")].values]
rownames = [f'stage_max_{i}' for i in [3019,3027,3028,3029,3030,3035,3041,3045,3050,3230,3036]]
# позиция целевых гидропостов
# positions = [i for i, x in enumerate(corr.columns.str.contains("3019|3027|3028|3029|3030|3035|3041|3045|3050|3230|3036]")) if x]
# print(positions)
corr.to_excel('results/lena_kirensk_corr.xlsx')
corr.loc[rownames, :].transpose().style.background_gradient(cmap='coolwarm')
# corr.style.background_gradient(cmap='coolwarm')

In [8]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])
        
    return np.array(data), np.array(labels)


def make_dataset(df, frac, target_label):
    test_size = len(df) - int(len(df) * frac)
    print('Train: ', test_size)
    print('Test: ', len(df) - test_size)
    dataset = df.values
    data_mean = np.nanmean(dataset[:test_size], axis=0)
    data_std = np.nanstd(dataset[:test_size], axis=0)
    print(dataset[:test_size].shape)
#     print(np.round(data_mean, 3), np.round(data_std, 2))
    dataset = (dataset-data_mean)/data_std
#     print(dataset[:,df.columns.get_loc(target_label)])
    # shaping training and test dataset to chunks
    x_train_multi, y_train_multi = multivariate_data(dataset[:, :], dataset[:, df.columns.get_loc(target_label)], len(dataset) - test_size,
                                                     None, past_history,
                                                     future_target, STEP)
    x_val_multi, y_val_multi = multivariate_data(dataset[:, :], dataset[:, df.columns.get_loc(target_label)],
                                                 0, int(len(df) * frac), past_history,
                                                 future_target, STEP)
    print('\n Input training data shape:')
    print(x_train_multi.shape[-2:], y_train_multi.shape[-2:])
    print('\n Input test data shape:')
    print(x_val_multi.shape[-2:], y_val_multi.shape[-2:])
#     print('\n Single step history:')
#     print(x_train_multi[0])
#     print('\n Single step target:')
#     print(y_train_multi[0])
    return x_train_multi, y_train_multi, x_val_multi, y_val_multi, data_mean, data_std


def model_build(df, target_label):
    print(target_label)
    x_train_multi, y_train_multi, x_val_multi, y_val_multi, data_mean, data_std = make_dataset(df, 0.2)
    
    # create train and test data from chunks
    train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
    val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))

    # training and test data shuffling
    train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

    multi_step_model = tf.keras.models.Sequential()
    multi_step_model.add(tf.keras.layers.LSTM(50,
                                              return_sequences=False,
                                              input_shape=x_train_multi.shape[-2:]))
    multi_step_model.add(tf.keras.layers.Dense(7))

    multi_step_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, 
                                                                beta_2=0.999, epsilon=1e-07, amsgrad=False,
                                                                name='Adam'), 
                             loss='mse')

    model_callbacks = [cb.EarlyStopping(patience=10), 
                              cb.ModelCheckpoint(filepath=f"output/"+ river + "_" + target_label + "_LSTM.h5", save_best_only=True)]
    multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=10,
                                          callbacks=model_callbacks)
    loss = multi_step_history.history['loss']
    val_loss = multi_step_history.history['val_loss']
    epochs = range(len(loss)) 
    plt.figure(figsize=[10,10])
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.legend()
    plt.show()

In [6]:
# define train/test split
target_labels = [f'stage_max_{i}' for i in [3019,3027,3028,3029,3030,3035,3041,3045,3050,3230,3036]]
print(target_labels)
# lookback parameters
past_history = 60 # how many instances a model sees for training, in compliance with other models' input
future_target = 7 # how many instances form a prediction step
STEP = 1 # step size - 1 for daily data
BATCH_SIZE = 30 
BUFFER_SIZE = 1000
EVALUATION_INTERVAL = len(df.values)/BATCH_SIZE # what data model sees for training
EPOCHS = 100 # how many times a model sees the data

['stage_max_3019', 'stage_max_3027', 'stage_max_3028', 'stage_max_3029', 'stage_max_3030', 'stage_max_3035', 'stage_max_3041', 'stage_max_3045', 'stage_max_3050', 'stage_max_3230', 'stage_max_3036']


In [ ]:
# run model training and saving
for l in target_labels:
    model_build(df, l)

In [29]:
# new data
new_df = pd.read_csv('data/lena/df_input_cp3.txt', index_col='date')
new_df.index = pd.DatetimeIndex(new_df.index)
new_df = new_df.dropna(axis=1, how='all')
del new_df['stage_max_3048']
print(new_df.shape)
new_df = new_df.apply(lambda x: x.fillna(x.mean()),axis=0)
new_df = new_df.apply(lambda x: x.fillna(0),axis=0)
# print(new_df.isna().sum())
ondates = pd.date_range(start=min(new_df.index), end=max(new_df.index), freq='D')



(12361, 191)
DatetimeIndex(['1985-01-01', '1985-01-02', '1985-01-03', '1985-01-04',
               '1985-01-05', '1985-01-06', '1985-01-07', '1985-01-08',
               '1985-01-09', '1985-01-10',
               ...
               '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25',
               '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', length=12783, freq='D')


In [9]:
# load trained model for time saving
output_df = pd.DataFrame() # df for all gauges' predictions
# gauges loop
for lab in [f'stage_max_{i}' for i in [3019,3027,3028,3029,3030,3035,3041,3045,3050,3230,3036]]:
    print(lab)
    # shape new data
    val_frac = 0.01
    nt, nv, vt, vv, dm, dstd = make_dataset(new_df, val_frac, lab)
#     print(nt.shape, nv.shape, vt.shape, vv.shape)
    # load model
    multi_step_model = load_model(f"output/lena_" + lab + "_LSTM.h5")
#     print(multi_step_model.summary())
#     print('Multi-step model prediction shape:')
#     print(multi_step_model.predict(nt).shape)
    # predict
    pred = pd.DataFrame(multi_step_model.predict(nt) * dstd[df.columns.get_loc(lab)] + dm[df.columns.get_loc(lab)])
#     print(pred.head())
#     for i in range(future_target):
#         pred[i] = pred[i].shift(i)
#     pred.columns = [f'pred_{i}' for i in range(1, future_target + 1)]
    # get time index
    start_index = len(new_df) - (len(new_df) - int(len(new_df) * val_frac)) + past_history + future_target
    pred['date'] = new_df[start_index:].index.values
    
    # 7-day forecast
#     forecast = pred.melt(id_vars = 'date_issued', var_name = 'horizon')
#     print(forecast.head())
#     forecast['date_valid'] = forecast['date_issued'] + pd.to_timedelta(forecast['horizon'].astype(int), unit='day')
#     forecast = forecast.sort_values(by=['date_issued', 'horizon'])
#     forecast.reset_index()
#     print(forecast.head(25))
    
    # get observations
#     pred['fact'] = new_df[start_index:][lab]
    
    # plot    
#     fig, ax = plt.subplots()
#     subdf = pred['2013-04-01':'2013-06-30']
#     subdf.plot(ax=ax)
#     subdf['fact'].plot(ax=ax, lw=5)
#     subdf.diff().plot()
#     subdf['fact'].diff().plot(lw=5)
#     plt.show()
#     subdf.head(7)
    pred['index'] = lab
    output_df = output_df.append(pred)
print(output_df.describe())

stage_max_3019
Train:  12238
Test:  123
(12238, 191)

 Input training data shape:
(60, 191) (12171, 7)

 Input test data shape:
(60, 191) (63, 7)
stage_max_3027
Train:  12238
Test:  123
(12238, 191)

 Input training data shape:
(60, 191) (12171, 7)

 Input test data shape:
(60, 191) (63, 7)
stage_max_3028
Train:  12238
Test:  123
(12238, 191)

 Input training data shape:
(60, 191) (12171, 7)

 Input test data shape:
(60, 191) (63, 7)
stage_max_3029
Train:  12238
Test:  123
(12238, 191)

 Input training data shape:
(60, 191) (12171, 7)

 Input test data shape:
(60, 191) (63, 7)
stage_max_3030
Train:  12238
Test:  123
(12238, 191)

 Input training data shape:
(60, 191) (12171, 7)

 Input test data shape:
(60, 191) (63, 7)
stage_max_3035
Train:  12238
Test:  123
(12238, 191)

 Input training data shape:
(60, 191) (12171, 7)

 Input test data shape:
(60, 191) (63, 7)
stage_max_3041
Train:  12238
Test:  123
(12238, 191)

 Input training data shape:
(60, 191) (12171, 7)

 Input test data sha

In [14]:
# output_df['index'] = output_df['index'].str[-4:].astype(int)
output_df.head(10)

,0,1,2,3,4,5,6,date,index
0,195.587860,185.976761,179.276459,182.952957,174.098526,159.069214,137.992126,1985-07-10,3019
1,187.247101,173.931778,164.207977,165.996307,156.010864,140.996155,121.002525,1985-07-11,3019
2,168.443100,156.803665,146.201691,146.987610,139.482758,123.754486,105.276970,1985-07-12,3019
3,160.241333,147.517334,135.260162,136.399902,131.309906,117.281456,103.222618,1985-07-13,3019
4,142.389099,132.688675,125.550392,132.286407,133.480270,124.539215,114.573914,1985-07-14,3019
5,113.926498,112.758057,111.733566,116.487686,111.656586,102.422043,88.550079,1985-07-15,3019
6,112.534378,107.726677,104.368118,108.001282,109.419922,105.342888,98.924240,1985-07-16,3019
7,116.470993,110.143631,103.238663,101.764801,107.399681,99.971146,97.193527,1985-07-17,3019
8,89.000702,101.676331,105.514816,102.921867,108.578064,99.181610,90.747696,1985-07-18,3019
9,91.029594,85.234329,82.614563,73.660126,75.546692,66.307106,68.482674,1985-07-19,3019


In [22]:
# extract test periods' predictions
test_cp1 = pd.read_csv(r'data/lena/test_cp4.csv')
test_cp1['date'] = pd.to_datetime(test_cp1['date'])
print(test_cp1.head())
solution_cp1 = pd.merge(test_cp1, output_df, how = 'left', left_on=['date', 'station_id'], right_on=['date', 'index'])
print(solution_cp1.head(10))
solution_cp1.to_excel('results/solcp1.xlsx')

   year  station_id  month  day       date  delta_stage_max
0  1993        3019      4  111 1993-04-21              NaN
1  1993        3019      4  112 1993-04-22              NaN
2  1993        3019      4  113 1993-04-23              NaN
3  1993        3019      4  114 1993-04-24              NaN
4  1993        3019      4  115 1993-04-25              NaN
   year  station_id  month  day       date  delta_stage_max   0   1   2   3  \
0  1993        3019      4  111 1993-04-21              NaN NaN NaN NaN NaN   
1  1993        3019      4  112 1993-04-22              NaN NaN NaN NaN NaN   
2  1993        3019      4  113 1993-04-23              NaN NaN NaN NaN NaN   
3  1993        3019      4  114 1993-04-24              NaN NaN NaN NaN NaN   
4  1993        3019      4  115 1993-04-25              NaN NaN NaN NaN NaN   
5  1993        3019      4  116 1993-04-26              NaN NaN NaN NaN NaN   
6  1993        3019      4  117 1993-04-27              NaN NaN NaN NaN NaN   
7  1997 